In [1]:
import requests
from urllib.parse import quote

client_id = "SWe77FM8c_EjAm5Y93h5"
client_secret = "mAz_OvLf7T"

def naver_shop_search(query, display=10):
    encoded_query = quote(query)
    url = f"https://openapi.naver.com/v1/search/shop.json?query={encoded_query}&display={display}&start=1&sort=sim&exclude=used:cbshop:rental"

    headers = {
        "X-Naver-Client-Id": client_id,
        "X-Naver-Client-Secret": client_secret
    }

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        return response.json()
    else:
        return {"error": response.status_code}

result = naver_shop_search("영양제", 100)

# if 'items' in result:
#   for item in result['items']:
#     if item['brand'] != '':
#       print(f"브랜드: {item['brand']}")
#       print(f"이름: {item['title']}")
#       print(f"상품ID: {item['productId']}")
#       print(f"링크: {item['link']}")
#       print("-" * 50)
# else:
#   print("Error:", result.get('error', 'Unknown error'))


In [2]:
!pip install -q chromadb sentence-transformers
!pip install -q langchain langchain-community langchain-core langchain_openai

In [3]:
import chromadb
from chromadb.utils import embedding_functions
from sentence_transformers import SentenceTransformer

# 1. 데이터 전처리 함수
def preprocess_items(items):
    processed = []
    for item in items:
        data = {
            "id": item['productId'],
            "text": f"{item['title']} {item['brand']} {item['maker']} {item['category3']}",  # 임베딩용 텍스트
            "metadata": {
                "price": int(item['lprice']),
                "brand": item['brand'],
                "category": f"{item['category1']}>{item['category2']}>{item['category3']}",
                "link": item['link']
            }
        }
        processed.append(data)
    return processed

# 2. Chroma DB 설정
client = chromadb.PersistentClient(path="./naver_shopping_db")
embedding_model = embedding_functions.SentenceTransformerEmbeddingFunction(
    model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
)

# 기존 컬렉션이 있으면 삭제 후 재생성
try:
    client.delete_collection("products")
except:
    pass
collection = client.create_collection(
    name="products",
    embedding_function=embedding_model,  # 반드시 임베딩 함수 재지정
    metadata={"hnsw:space": "cosine"}  # 거리 측정 방식 변경
)

# 3. 데이터 삽입 (API 결과 사용)
if 'items' in result:
    processed_data = preprocess_items(result['items'])

    # 벡터 DB에 문서 추가
    collection.add(
        ids=[item['id'] for item in processed_data],
        documents=[item['text'] for item in processed_data],
        metadatas=[item['metadata'] for item in processed_data]
    )
    print("✅ 성공적으로", len(processed_data), "개 상품 저장됨")
else:
    print("Error:", result.get('error', '데이터 없음'))

# 4. 검색 예시 함수
def search_products(query, n_results=3):
    results = collection.query(
        query_texts=[query],
        n_results=n_results
    )
    for idx, (doc, meta) in enumerate(zip(results['documents'][0], results['metadatas'][0]), 1):
        print(f"🔍 [{idx}] {doc}")
        print(f"   가격: {meta['price']:,}원 | 브랜드: {meta['brand']}")
        print(f"   카테고리: {meta['category']}")
        print(f"   링크: {meta['link']}")
        print("-"*50)

✅ 성공적으로 100 개 상품 저장됨


In [4]:
# LLM 적용 전, 벡터 DB에 잘 들어가있는지 확인
search_products("피곤할 때 먹는 영양제", 5)

🔍 [1] 얼라이브 원스데일리 멀티/포맨/ 우먼 60정 비타민B군 미네랄 종합<b>영양제</b> 네츄럴라이프얼라이브 네이쳐스웨이 비타민제
   가격: 23,900원 | 브랜드: 네츄럴라이프얼라이브
   카테고리: 식품>건강식품>비타민제
   링크: https://smartstore.naver.com/main/products/309612084
--------------------------------------------------
🔍 [2] 베로카 발포비타민 60정 멀티 종합 비타민 <b>영양제</b> 물에타먹는 마시는 에너지 코스트코 베로카  비타민제
   가격: 23,700원 | 브랜드: 베로카
   카테고리: 식품>건강식품>비타민제
   링크: https://smartstore.naver.com/main/products/9172711926
--------------------------------------------------
🔍 [3] 뉴트리포유 듀얼 이뮨샷 액상 마시는 멀티비타민 미네랄 <b>영양제</b> 30개입 피로회복 종합<b>영양제</b>  소셜빈 비타민제
   가격: 54,900원 | 브랜드: 
   카테고리: 식품>건강식품>비타민제
   링크: https://smartstore.naver.com/main/products/10532848731
--------------------------------------------------
🔍 [4] 센트룸 실버 포 맨 종합 멀티 비타민 112정 코스트코 남성 50+ 대용량 <b>영양제</b> 센트룸 화이자 비타민제
   가격: 34,900원 | 브랜드: 센트룸
   카테고리: 식품>건강식품>비타민제
   링크: https://smartstore.naver.com/main/products/9244093197
--------------------------------------------------
🔍 [5] 세노비스 수험생 중학생 고등학생 성인 청소년 종합 <b>영양제</b> 멀티비타민 미네랄 젤리

In [10]:
# Hugging Face 및 Google API 인증
from google.colab import userdata
#HF_TOKEN = ''
#GOOGLE_API_KEY = ''
#GOOGLE_CSE_ID = ''
#OPENAI_API_KEY = ''

In [11]:
# 2. Google Programmable Search Engine 설정
from googleapiclient.discovery import build

def google_search(query, num=3):
    """
    구글 커스텀 검색을 실행하는 함수

    Parameters:
    - query (str): 검색할 질의어(검색어)입니다. 예: "비타민C 효능"
    - num (int): 검색 결과에서 반환할 최대 문서(스니펫) 개수입니다. 기본값은 3개입니다.

    Returns:
    - list: 각 검색 결과의 요약(snippet) 텍스트 리스트
    """
    service = build("customsearch", "v1", developerKey=GOOGLE_API_KEY)
    res = service.cse().list(q=query, cx=GOOGLE_CSE_ID, num=num).execute()
    return [item["snippet"] for item in res["items"]]


In [12]:
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.tools import tool

# 2. 검색 도구 정의 (Function Calling)
@tool
def search_products(query: str, max_results: int = 5) -> list:
    """제품 검색을 위해 Chroma DB를 쿼리하는 도구"""
    """
    Chroma DB를 쿼리하여 제품 정보를 검색하는 도구

    Parameters:
    - query (str): 제품 검색 질의어입니다. 예: "비타민C 고함량"
    - max_results (int): 반환할 최대 제품 개수입니다. 기본값은 3개입니다.

    Returns:
    - list: 각 제품의 정보(제품명, 가격, 링크)를 담은 딕셔너리들의 리스트
      예시: [{"제품": "제품명", "가격": 10000, "링크": "https://..."}, ...]
    """
    results = collection.query(
        query_texts=[query],
        n_results=max_results
    )
    return [{
        "제품": doc,
        "가격": meta['price'],
        "링크": meta['link']
    } for doc, meta in zip(results['documents'][0], results['metadatas'][0])]

@tool
def google_search_tool(query: str, num_results: int = 10) -> list:
    """구글 검색을 통해 제품 상세 정보 및 최신 논문을 찾는 도구. 질문 답변에 필요한 웹 기반 정보 수집에 사용"""
    """
    구글 검색 API를 통해 웹에서 정보를 검색하는 도구

    Parameters:
    - query (str): 검색할 질의어(검색어)입니다. 예: "2025년 영양제 트렌드"
    - num_results (int): 반환할 최대 검색 결과 개수입니다. 기본값은 3개입니다.

    Returns:
    - list: 각 검색 결과의 요약(snippet) 텍스트 리스트
    """
    return google_search(query, num=num_results)

# 3. 에이전트 설정
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3, openai_api_key=OPENAI_API_KEY)
llm_with_tools = llm.bind_tools([search_products, google_search_tool])

system_prompt = """당신은 영양제 전문 약사 어시스턴트입니다. 다음 규칙을 준수하세요:
- 사용자 질문 유형에 따라 적절한 도구 선택
  - 제품 검색: search_products
  - 제품 상세 정보, 최신 논문: google_search_tool
- 검색 결과를 바탕으로 정확한 정보 제공
- 추천하는 이유를 자세하게 설명
- 가격, 성분 비교시 표 형태로 응답
- 모르는 정보는 확대 해석하지 말 것
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}"),
    MessagesPlaceholder("agent_scratchpad")
])

tools = [search_products]
agent = create_openai_tools_agent(llm_with_tools, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False) # verbose=True 로 변경안해도 됨


In [13]:
# 4. 실행 예시
response = agent_executor.invoke({
    "input": "종종 머리가 아프고 눈이 피곤해. 어떤 영양제를 먹으면 좋을지 추천해줘. 주요 성분과 함량도 알려줘."
})
print(response['output'])

다음은 두통과 눈 피로에 도움이 될 수 있는 영양제 추천 목록입니다. 각 제품의 주요 성분과 가격을 포함하였습니다.

| 제품명 | 주요 성분 | 가격 | 링크 |
| --- | --- | --- | --- |
| 종근당 아임비타 멀티비타민 이뮨플러스 14병 샷 | 비타민 A, C, D, E, B군, 아연, 셀레늄 | 21,900원 | [구매하기](https://smartstore.naver.com/main/products/11097440087) |
| [1+1]종근당 아임비타 이뮨플러스 샷 2박스 | 비타민 A, C, D, E, B군, 아연, 셀레늄 | 23,500원 | [구매하기](https://smartstore.naver.com/main/products/8566190007) |
| 뉴트리포유 듀얼 이뮨샷 액상 | 비타민 A, C, D, E, B군, 미네랄 (철, 아연) | 54,900원 | [구매하기](https://smartstore.naver.com/main/products/10532848731) |
| 세노비스 수험생 종합 영양제 | 비타민 A, C, D, E, B군, 미네랄 (철, 아연) | 23,900원 | [구매하기](https://smartstore.naver.com/main/products/7281402394) |
| 독일 비타하임 발포비타민 | 비타민 C, B군 | 2,700원 | [구매하기](https://smartstore.naver.com/main/products/136290431) |

### 추천 이유
1. **비타민 B군**: 두통과 눈 피로를 완화하는 데 도움을 줄 수 있습니다. 특히 B2(리보플라빈)와 B6(피리독신)는 에너지 대사에 중요한 역할을 합니다.
2. **비타민 C**: 항산화 작용이 있어 피로 회복에 도움을 줄 수 있습니다.
3. **아연과 셀레늄**: 면역력 증진과 피로 회복에 기여합니다.

이 제품들은 다양한 비타민과 미네랄을 포함하고 있어 종합적으로 건강을 지원할 수 있습니다. 필요에 따라 선택하시면 좋겠

# 김지은_아래의 두개 코드를 봐주세요! [15], [31]

In [15]:
!pip install ipywidgets
import sys
!{sys.executable} -m pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
Paths used for configuration of notebook: 
    	/root/.jupyter/nbconfig/notebook.json
Paths used for configuration of notebook: 
    	
      - Validating: OK
Paths used for configuration of notebook: 
    	/root/.jupyter/nbconfig/notebook.json


In [47]:
# 설치 필요: !pip install -q openai ipywidgets requests

import openai
import ipywidgets as widgets
from IPython.display import display, clear_output
import requests
from urllib.parse import quote
import json
import re

# ✅ OpenAI 초기화
from openai import OpenAI
client = OpenAI(api_key="")  # 본인 키 입력

# ✅ 네이버 API
naver_client_id = "SWe77FM8c_EjAm5Y93h5"
naver_client_secret = "mAz_OvLf7T"

# 🔍 네이버 쇼핑 API
def naver_shop_search(query, display=5):
    encoded = quote(query)
    url = f"https://openapi.naver.com/v1/search/shop.json?query={encoded}&display={display}"
    headers = {
        "X-Naver-Client-Id": naver_client_id,
        "X-Naver-Client-Secret": naver_client_secret
    }
    res = requests.get(url, headers=headers)
    return res.json().get("items", [])

# 🔐 필터링
def is_valid_product(item):
    title = re.sub(r"<.*?>", "", item['title']).lower()
    banned_keywords = [
        '시약', '실험', '키트', '화공', '공업', '가성소다',
        '염화칼슘', '알긴산나트륨', '수산화나트륨', '티오황산',
        '오호', '제조용', 'chemical', 'lab', '실험실', 'experiment'
    ]
    return not any(bad_kw in title for bad_kw in banned_keywords)

# 🧠 Function Calling 정의
function_def = {
    "name": "recommend_nutrients",
    "description": "증상에 따라 추천 또는 제거해야 할 성분 목록 반환",
    "parameters": {
        "type": "object",
        "properties": {
            "nutrients": {
                "type": "array",
                "items": {
                    "type": "object",
                    "properties": {
                        "nutrient": {"type": "string"},
                        "action": {"type": "string", "enum": ["추천", "제거"]}
                    },
                    "required": ["nutrient", "action"]
                }
            }
        },
        "required": ["nutrients"]
    }
}

def get_nutrients_action(symptom, gender, age, pregnancy, mode):
    prompt = f"""
    사용자 정보:
    - 성별: {gender}
    - 나이: {age}
    - 임신 여부: {pregnancy}
    - 선택 기능: {mode}
    증상: {symptom}

    부족한 성분은 '추천', 과다 위험 성분은 '제거'로 JSON 배열 반환
    """
    res = client.chat.completions.create(
        model="gpt-4-0613",
        messages=[
            {"role": "system", "content": "너는 건강 기반의 영양제 조언 전문가야."},
            {"role": "user", "content": prompt}
        ],
        functions=[function_def],
        function_call={"name": "recommend_nutrients"},
        temperature=0.5
    )
    args = res.choices[0].message.function_call.arguments
    parsed = json.loads(args)
    return parsed.get("nutrients", [])

# --- ✅ UI 구성 ---
gender = widgets.ToggleButtons(options=["남성", "여성"], description='성별:')
age = widgets.IntText(value=30, description='만 나이:')
pregnancy = widgets.ToggleButtons(options=["해당 없음", "임신 중", "임신 가능성"], description='임신 여부:')

# ✅ 메뉴 버튼 너비 + 줄바꿈 허용
mode = widgets.ToggleButtons(
    options=[
        "식습관 분석을 통한 영양제 추천",
        "불편 현상에 따른 영양제 추천",
        "영양제 과다 복용 확인"
    ],
    description='메뉴:',
    layout=widgets.Layout(width='100%', flex_flow='row wrap', justify_content='flex-start'),
    style={'button_width': 'auto'}
)

symptom_input = widgets.Text(
    placeholder='증상 또는 복용 제품 입력',
    description='입력:',
    layout=widgets.Layout(width='95%')
)

submit_button = widgets.Button(description="제출", button_style="success")
output_box = widgets.Output()

# --- 실행 함수 ---
def on_submit_clicked(b):
    with output_box:
        clear_output()
        info = {
            "성별": gender.value,
            "나이": age.value,
            "임신 여부": pregnancy.value,
            "선택 기능": mode.value
        }
        symptom = symptom_input.value.strip()

        print(f"사용자 정보: {info}")
        print(f"선택한 기능: {mode.value}\n")

        if not symptom:
            print("⚠️ 증상이나 복용 제품을 입력해주세요.")
            return

        nutrients = get_nutrients_action(symptom, gender.value, age.value, pregnancy.value, mode.value)

        if not nutrients:
            print("⚠️ 성분 추천에 실패했습니다.")
            return

        for item in nutrients:
            nutrient, action = item['nutrient'], item['action']

            if action == "추천":
                print(f"🧾 '{symptom}'에 대한 AI 추천 결과:")
                print(f"- {nutrient} 성분이 포함된 제품이 적합할 수 있습니다.\n")
            elif action == "제거":
                print(f"⚠️ '{nutrient}' 성분이 중복 또는 과다 복용 위험이 있습니다.")
                print("🔍 해당 성분이 포함된 제품을 확인하고 복용 여부를 재검토하세요.\n")

            products = naver_shop_search(nutrient)
            shown = False
            for p in products:
                if not is_valid_product(p):
                    continue
                title = re.sub(r"<.*?>", "", p['title'])
                print(f"• {title}")
                print(f"  - 브랜드: {p.get('brand', '정보 없음')}")
                print(f"  - 가격: {int(p['lprice']):,}원")
                print(f"  - 링크: {p['link']}\n")
                shown = True
            if not shown:
                print("🔍 관련된 정상 제품이 검색되지 않았습니다.\n")

        print("※ 이 답변은 빅데이터 기반으로 제공되며, 정확한 복약은 전문가 상담이 필요합니다.")

# --- UI 실행 ---
submit_button.on_click(on_submit_clicked)
display(widgets.VBox([
    gender, age, pregnancy, mode, symptom_input, submit_button, output_box
]))


# **지은_"영양제 과다 복용 확인"까지 되는 코드입니다!! 확인 부탁드려요**

In [53]:
# 설치 필요: !pip install -q openai ipywidgets requests

import openai
import ipywidgets as widgets
from IPython.display import display, clear_output
import requests
from urllib.parse import quote
import json
import re

# ✅ OpenAI 초기화
from openai import OpenAI
client = OpenAI(api_key="")  # ← 본인 키 입력

# ✅ 네이버 API
naver_client_id = "SWe77FM8c_EjAm5Y93h5"
naver_client_secret = "mAz_OvLf7T"

def naver_shop_search(query, display=5):
    encoded = quote(query)
    url = f"https://openapi.naver.com/v1/search/shop.json?query={encoded}&display={display}"
    headers = {
        "X-Naver-Client-Id": naver_client_id,
        "X-Naver-Client-Secret": naver_client_secret
    }
    res = requests.get(url, headers=headers)
    return res.json().get("items", [])

def is_valid_product(item):
    title = re.sub(r"<.*?>", "", item['title']).lower()
    banned_keywords = [
        '시약', '실험', '키트', '화공', '공업', '가성소다',
        '염화칼슘', '알긴산나트륨', '수산화나트륨', '티오황산',
        '오호', '제조용', 'chemical', 'lab', '실험실', 'experiment'
    ]
    return not any(bad_kw in title for bad_kw in banned_keywords)

function_def = {
    "name": "recommend_nutrients",
    "description": "과다 복용 시 제거할 성분 및 제품 목록 반환",
    "parameters": {
        "type": "object",
        "properties": {
            "nutrients": {
                "type": "array",
                "items": {
                    "type": "object",
                    "properties": {
                        "nutrient": {"type": "string"},
                        "action": {"type": "string", "enum": ["제거"]},
                        "exclude_products": {"type": "array", "items": {"type": "string"}},
                        "estimated_amount": {"type": "string"},
                        "reason": {"type": "string"}
                    },
                    "required": ["nutrient", "action"]
                }
            }
        },
        "required": ["nutrients"]
    }
}

def get_nutrients_action(symptom, gender, age, pregnancy, mode):
    prompt = f"""
    사용자 정보:
    - 성별: {gender}
    - 나이: {age}
    - 임신 여부: {pregnancy}
    - 선택 기능: {mode}
    복용 중인 영양제 또는 증상: {symptom}

    과다 복용 확인 기능인 경우, 복용 중인 영양제의 주요 성분 중 중복 또는 과다한 성분을 찾아 '제거'로 표시하고,
    제외할 제품명을 'exclude_products' 항목에 배열로 담고, 현재 복용 중인 추정 총량과 과다 복용 이유도 포함해줘.
    추천 성분은 생략. JSON 형식의 Function Call로만 반환.
    """
    res = client.chat.completions.create(
        model="gpt-4-0613",
        messages=[
            {"role": "system", "content": "너는 건강 기반의 영양제 조언 전문가야."},
            {"role": "user", "content": prompt}
        ],
        functions=[function_def],
        function_call={"name": "recommend_nutrients"},
        temperature=0.5
    )
    args = res.choices[0].message.function_call.arguments
    parsed = json.loads(args)
    return parsed.get("nutrients", [])

# --- UI 구성 ---
gender = widgets.ToggleButtons(options=["남성", "여성"], description='성별:')
age = widgets.IntText(value=30, description='만 나이:')
pregnancy = widgets.ToggleButtons(options=["해당 없음", "임신 중", "임신 가능성"], description='임신 여부:')
mode = widgets.ToggleButtons(
    options=[
        "식습관 분석을 통한 영양제 추천",
        "불편 현상에 따른 영양제 추천",
        "영양제 과다 복용 확인"
    ],
    description='메뉴:',
    layout=widgets.Layout(width='100%', flex_flow='row wrap'),
    style={'button_width': 'auto'}
)
symptom_input = widgets.Text(
    placeholder='증상 또는 복용 제품 입력',
    description='입력:',
    layout=widgets.Layout(width='95%')
)
submit_button = widgets.Button(description="제출", button_style="success")
output_box = widgets.Output()

# --- 실행 함수 ---
def on_submit_clicked(b):
    with output_box:
        clear_output()
        info = {
            "성별": gender.value,
            "나이": age.value,
            "임신 여부": pregnancy.value,
            "선택 기능": mode.value
        }
        symptom = symptom_input.value.strip()
        print(f"사용자 정보: {info}")
        print(f"선택한 기능: {mode.value}\n")

        if not symptom:
            print("⚠️ 증상이나 복용 제품을 입력해주세요.")
            return

        nutrients = get_nutrients_action(symptom, gender.value, age.value, pregnancy.value, mode.value)
        if not nutrients:
            print("⚠️ 성분 분석에 실패했습니다.")
            return

        for item in nutrients:
            if item["action"] != "제거":
                continue
            nutrient = item['nutrient']
            print(f"⚠️ '{nutrient}' 성분이 과다 복용 위험이 있습니다.")
            if "estimated_amount" in item:
                print(f"📌 현재 복용 추정량: {item['estimated_amount']}")
            if "reason" in item:
                print(f"🧾 사유: {item['reason']}")
            if "exclude_products" in item and item["exclude_products"]:
                print("📦 다음 제품을 제외하면 적정 섭취로 조절할 수 있습니다:")
                for prod in item["exclude_products"]:
                    print(f"  - {prod}")
            print()

# UI 실행
submit_button.on_click(on_submit_clicked)
display(widgets.VBox([gender, age, pregnancy, mode, symptom_input, submit_button, output_box]))
